# RGB and RGB-D Fusion
> borrow code from https://github.com/jiangyao-scu/JL-DCF-pytorch and add some tips
- toc: true 
- badges: true
- comments: true
- sticky_rank: 2
- author: Bowen
- categories: [pytorch, fastai,fusion]


![](https://i.loli.net/2021/12/03/nHjQ158cRYxFCAZ.png)

In [1]:
import mmcv
import matplotlib.pyplot as plt
from fastcore.basics import *
from fastai.vision.all import *
from fastai.torch_basics import *
import warnings
warnings.filterwarnings("ignore")
import kornia
from kornia.constants import Resample
from kornia.color import *
from kornia import augmentation as K
import kornia.augmentation as F
import kornia.augmentation.random_generator as rg
from torchvision.transforms import functional as tvF
from torchvision.transforms import transforms
from torchvision.transforms import PILToTensor
from functools import partial
from timm.models.layers import trunc_normal_, DropPath
from timm.data import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD
from timm.models.vision_transformer import _cfg
from einops import rearrange
from timm.models.registry import register_model

set_seed(105)

## dataset overview

In [12]:
rootPath = Path('/home/ubuntu/sharedData/swp/dlLab/fastaiRepository/fastai/data/SOD/RGBDcollection/')
rgbPath = Path('/home/ubuntu/sharedData/swp/dlLab/fastaiRepository/fastai/data/SOD/RGBDcollection/LR/')
depPath = Path('/home/ubuntu/sharedData/swp/dlLab/fastaiRepository/fastai/data/SOD/RGBDcollection/depth/')
gtPath = Path('/home/ubuntu/sharedData/swp/dlLab/fastaiRepository/fastai/data/SOD/RGBDcollection/GT/')


In [14]:
rgbFiles = get_image_files(rgbPath)
depFiles = get_image_files(depPath)
lblFiles = get_image_files(gtPath)
rgbFiles[0]
depFiles[0]
lblFiles[0]

Path('/home/ubuntu/sharedData/swp/dlLab/fastaiRepository/fastai/data/SOD/RGBDcollection/LR/001477_left_1_ori.jpg')

Path('/home/ubuntu/sharedData/swp/dlLab/fastaiRepository/fastai/data/SOD/RGBDcollection/depth/10_01-16-36_0_Depth.png')

Path('/home/ubuntu/sharedData/swp/dlLab/fastaiRepository/fastai/data/SOD/RGBDcollection/GT/9_07-38-26_0_GT.png')

In [16]:
to_tensor = transforms.ToTensor()
to_pil = transforms.ToPILImage()
rgbImage = Image.open(rgbFiles[0])
lblImage = Image.open(lblFiles[0])
depImage = Image.open(depFiles[0])
rgbTensor = image2tensor(rgbImage)
lblTensor = image2tensor(lblImage)
depTensor = image2tensor(depImage)

In [20]:
torch.unique(depTensor)
len(torch.unique(depTensor))

tensor([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
         28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
         42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
         56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
         70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
         84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
         98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
        112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
        126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139,
        140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153,
        154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167,
        168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 1

256

In [40]:
import cv2
def Normalization(image):
    set_trace()
    in_ = image[:, :, ::-1]
    in_ = in_ / 255.0
    in_ -= np.array((0.485, 0.456, 0.406))
    in_ /= np.array((0.229, 0.224, 0.225))
    return in_

In [39]:
temp = np.array((1,1,1),dtype=np.float32)
temp2 = np.array((2,2,2),dtype=np.float32)
temp-=temp2
temp/=temp2
temp

array([-0.5, -0.5, -0.5], dtype=float32)

In [41]:

def load_image(path,image_size):
    im = cv2.imread(path)
    in_ = np.array(im, dtype=np.float32)
    in_ = cv2.resize(in_, (image_size, image_size))
    in_ = Normalization(in_)
    return in_

In [28]:
rgbFiles[0]

Path('/home/ubuntu/sharedData/swp/dlLab/fastaiRepository/fastai/data/SOD/RGBDcollection/LR/001477_left_1_ori.jpg')

## slicing issues of three dimensions

In [46]:

import numpy as np
 
b = np.array([[[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12]],
              [[13, 14, 15, 16], [17, 18, 19, 20], [21, 22, 23, 24]],
              [[25, 26, 27, 28], [29, 30, 31, 32], [33, 34, 35, 36]],
              ])
 
print(f'b is {b}')
print(b.shape)
print("b[0, ::],b[1, ::],b[-1, ::],b[0:2, ::]")
print("b[0, ::]",b[0, ::], b[0, ::].shape)
print("b[1, ::]",b[1, ::], b[1, ::].shape)
print("b[-1, ::]",b[-1, ::], b[-1, ::].shape)
print("b[0:2, ::]",b[0:2, ::], b[0:2, ::].shape)
print("b[:, 0:],b[:, 1:],b[:, -1:],b[:, 0:2:]")
print("b[:, 0:]",b[:, 0:], b[:, 0:].shape)
print("b[:, 1:]",b[:, 1:], b[:, 1:].shape)
print("b[:, -1:]",b[:, -1:], b[:, -1:].shape)
print("b[:, 0:2:]",b[:, 0:2:], b[:, 0:2:].shape)
print("b[::, 0],b[::, 1],b[::, -1],b[::, 0:2:]")
print("b[::, 0]",b[::, 0], b[::, 0].shape)
print("b[::, 1]",b[::, 1], b[::, 1].shape)
print("b[::, -1]",b[::, -1], b[::, -1].shape)
print("b[::, 0:2:]",b[::, 0:2:], b[::, 0:2].shape)
print("b[:,:, 0],b[:,:, 1],b[:,:, -1],b[:,:, 0:2:]")
print("b[:, :, 0]",b[:, :, 0], b[:, :, 0].shape)
print("b[:, :, 1]",b[:, :, 1], b[:, :, 1].shape)
print("b[:, :, -1]",b[:, :, -1], b[:, :, -1].shape)


b is [[[ 1  2  3  4]
  [ 5  6  7  8]
  [ 9 10 11 12]]

 [[13 14 15 16]
  [17 18 19 20]
  [21 22 23 24]]

 [[25 26 27 28]
  [29 30 31 32]
  [33 34 35 36]]]
(3, 3, 4)
b[0, ::],b[1, ::],b[-1, ::],b[0:2, ::]
b[0, ::] [[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]] (3, 4)
b[1, ::] [[13 14 15 16]
 [17 18 19 20]
 [21 22 23 24]] (3, 4)
b[-1, ::] [[25 26 27 28]
 [29 30 31 32]
 [33 34 35 36]] (3, 4)
b[0:2, ::] [[[ 1  2  3  4]
  [ 5  6  7  8]
  [ 9 10 11 12]]

 [[13 14 15 16]
  [17 18 19 20]
  [21 22 23 24]]] (2, 3, 4)
b[:, 0:],b[:, 1:],b[:, -1:],b[:, 0:2:]
b[:, 0:] [[[ 1  2  3  4]
  [ 5  6  7  8]
  [ 9 10 11 12]]

 [[13 14 15 16]
  [17 18 19 20]
  [21 22 23 24]]

 [[25 26 27 28]
  [29 30 31 32]
  [33 34 35 36]]] (3, 3, 4)
b[:, 1:] [[[ 5  6  7  8]
  [ 9 10 11 12]]

 [[17 18 19 20]
  [21 22 23 24]]

 [[29 30 31 32]
  [33 34 35 36]]] (3, 2, 4)
b[:, -1:] [[[ 9 10 11 12]]

 [[21 22 23 24]]

 [[33 34 35 36]]] (3, 1, 4)
b[:, 0:2:] [[[ 1  2  3  4]
  [ 5  6  7  8]]

 [[13 14 15 16]
  [17 18 19 20]]

 [[25 

In [42]:
load_image('/home/ubuntu/sharedData/swp/dlLab/fastaiRepository/fastai/data/SOD/RGBDcollection/LR/001477_left_1_ori.jpg',512)

> /tmp/ipykernel_27971/2475151000.py(4)Normalization()
      2 def Normalization(image):
      3     set_trace()
----> 4     in_ = image[:, :, ::-1]
      5     in_ = in_ / 255.0
      6     in_ -= np.array((0.485, 0.456, 0.406))

ipdb> image
array([[[253.     , 254.     , 250.     ],
        [253.     , 254.     , 250.     ],
        [253.     , 254.     , 250.     ],
        ...,
        [233.48633, 227.48633, 220.48633],
        [229.1377 , 223.1377 , 216.1377 ],
        [229.     , 223.     , 216.     ]],

       [[253.     , 254.     , 250.     ],
        [253.     , 254.     , 250.     ],
        [253.     , 254.     , 250.     ],
        ...,
        [231.33528, 225.33528, 218.33528],
        [226.27832, 220.27832, 213.27832],
        [225.80833, 219.80833, 212.80833]],

       [[253.     , 254.     , 250.     ],
        [253.     , 254.     , 250.     ],
        [253.     , 254.     , 250.     ],
        ...,
        [227.54199, 221.54199, 214.54199],
        [221.57063, 215.57

array([[[2.1632845 , 2.4110646 , 2.6051416 ],
        [2.1632845 , 2.4110646 , 2.6051416 ],
        [2.1632845 , 2.4110646 , 2.6051416 ],
        ...,
        [1.6578702 , 1.9468895 , 2.265034  ],
        [1.5834007 , 1.8707579 , 2.189241  ],
        [1.5810429 , 1.8683474 , 2.186841  ]],

       [[2.1632845 , 2.4110646 , 2.6051416 ],
        [2.1632845 , 2.4110646 , 2.6051416 ],
        [2.1632845 , 2.4110646 , 2.6051416 ],
        ...,
        [1.6210339 , 1.9092311 , 2.2275429 ],
        [1.5344347 , 1.8206987 , 2.1394043 ],
        [1.5263865 , 1.8124709 , 2.131213  ]],

       [[2.1632845 , 2.4110646 , 2.6051416 ],
        [2.1632845 , 2.4110646 , 2.6051416 ],
        [2.1632845 , 2.4110646 , 2.6051416 ],
        ...,
        [1.5560747 , 1.842822  , 2.1614292 ],
        [1.4538167 , 1.7382812 , 2.0573533 ],
        [1.4344808 , 1.7185138 , 2.0376735 ]],

       ...,

       [[1.8711603 , 1.6397306 , 0.9134887 ],
        [1.9877274 , 1.7700812 , 0.9922714 ],
        [2.0413816 , 1